In [ ]:
# pip install sentence_transformers

In [10]:
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input, decode_predictions
from tensorflow.keras.applications.xception import Xception, preprocess_input, decode_predictions
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input, decode_predictions
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input, decode_predictions
from tensorflow.keras.applications.densenet import DenseNet121, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import numpy as np
import pandas as pd
import cv2
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer, util
from PIL import Image
import os
import tqdm

In [11]:
resnet50 = ResNet50(include_top=False, weights='imagenet', pooling='avg')
xception = Xception(include_top=False, weights='imagenet', pooling='avg')
vgg19 = VGG19(include_top=False, weights='imagenet', pooling='avg')
inceptionv3 = InceptionV3(include_top=False, weights='imagenet', pooling='avg')
densenet = DenseNet121(include_top=False, weights='imagenet', pooling='avg')
clip = SentenceTransformer('clip-ViT-B-32')

ftfy or spacy is not installed using BERT BasicTokenizer instead of ftfy.


In [12]:
def return_image_embedding(model,img_path):
    if model != clip:
        img = image.load_img(img_path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        preds = model.predict(x)
        curr_df = pd.DataFrame(preds[0])
        curr_df = list(curr_df[0])
    else:
        curr_df = list(clip.encode(Image.open(img_path)))
    return curr_df

In [13]:
def files(path):
    for file in os.listdir(path):
        if os.path.isfile(os.path.join(path, file)):
            yield file

In [ ]:
'''
img_path = train_img_path + "/" + img_list[0]
model = densenet
return_image_embedding(model,img_path)
'''

In [14]:
def get_img_encoding_file(train_img_path,count_img,model,model_name):
    i = 1
    file_num = 0
    list_img = []
    list_encodings = []


    for img in tqdm.tqdm(list(files(path = train_img_path))):

        img_path = train_img_path + "/" + img
        list_img.append(int(img[:-4]))
        list_encodings.append(return_image_embedding(model,img_path))

        if (i % count_img) == 0:
            df = pd.DataFrame(list_encodings)
            df['img'] = list_img
            df.columns = df.columns.map(str)

            file_name = "./img_encodings/train_img_encodings_" + f"{model_name}" + "_" + f"{file_num}" + ".parquet"

            df.to_parquet(file_name)  

            file_num = file_num + 1
            i = 1

            list_img = []
            list_encodings = []

        else:

            i = i + 1

In [ ]:
train_img_path = "./coco/images/train2017"
count_img = 10
model = densenet
model_name = 'densenet'

get_img_encoding_file(train_img_path,count_img,model,model_name)

In [25]:
'''
restored_df = pd.read_parquet(path = './img_encodings//train_img_encodings_densenet_0.parquet')
restored_df
'''